In [1]:
%cd "/content/drive/MyDrive/PhD/CE7455 - NLP/Project/text-to-anime"

/content/drive/MyDrive/PhD/CE7455 - NLP/Project/text-to-anime


In [2]:
%pip install -r requirements.txt

     |████████████████████████████████| 245kB 17.3MB/s 


In [3]:
import torch

from model import Tacotron2
from train import main
from utils import HParams

torch.backends.cudnn.enabled = True


params = HParams(
    n_mel_channels=204,
    max_decoder_steps=240,
    epochs=50,
    iters_per_checkpoint=45,
    learning_rate=2e-3,
    batch_size=8,
    fp16_run=True
)
main(params)

Downloading: "https://github.com/nvidia/DeepLearningExamples/archive/torchhub.zip" to /root/.cache/torch/hub/torchhub.zip


Epoch: 0
Train loss 1 20.584335 Grad Norm 4.549751 1.78s/it
Train loss 2 20.059137 Grad Norm 4.633248 1.10s/it
Train loss 3 7.257370 Grad Norm 1.862494 3.24s/it
Train loss 4 16.643414 Grad Norm 2.770836 2.29s/it
Train loss 5 7.671134 Grad Norm 1.830688 2.46s/it
Train loss 6 10.044757 Grad Norm 2.331043 3.21s/it
Train loss 7 5.908837 Grad Norm 1.307446 2.47s/it
Train loss 8 10.508549 Grad Norm 1.849625 2.60s/it
Train loss 9 13.652857 Grad Norm 2.343244 1.58s/it
Epoch: 1
Train loss 10 10.931065 Grad Norm 2.052390 2.10s/it
Train loss 11 10.486967 Grad Norm 1.681360 5.02s/it
Train loss 12 9.035996 Grad Norm 1.564785 3.25s/it
Train loss 13 14.118131 Grad Norm 2.454660 2.82s/it
Train loss 14 10.482509 Grad Norm 2.230345 3.77s/it
Train loss 15 6.553260 Grad Norm 1.205558 3.50s/it
Train loss 16 13.001539 Grad Norm 2.361248 2.77s/it
Train loss 17 10.569710 Grad Norm 2.253839 2.81s/it
Train loss 18 7.832305 Grad Norm 1.475716 2.56s/it
Epoch: 2
Train loss 19 6.167191 Grad Norm 1.549004 3.54s/it
T

In [4]:
import numpy as np
import torch

from model import Tacotron2
from text import text_to_sequence
from utils import HParams

model = Tacotron2(HParams(n_mel_channels=60, max_decoder_steps=240)).cuda()
checkpoint = torch.load("best-lips.pt", map_location="cpu")
model.load_state_dict(checkpoint["state_dict"])


def infer(text):
    sequence = text_to_sequence(text, ["english_cleaners"])
    sequence = torch.IntTensor(sequence)[None, :].cuda().long()
    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
    with open("output_mel.npy", "wb") as f:
        np.save(f, mel_outputs.float().data.cpu().numpy()[0])
    with open("output_post.npy", "wb") as f:
        np.save(f, mel_outputs_postnet.float().data.cpu().numpy()[0])
    return (mel_outputs, mel_outputs_postnet, alignments)


# infer("IT'S NOT JUST MY WORK")
infer("Hello World")

Warning! Reached max decoder steps


(tensor([[[-0.2879, -0.4858, -0.5473,  ..., -1.1656, -0.8704, -0.6747],
          [ 0.0226, -0.1612, -0.2008,  ..., -1.7950, -1.5669, -1.1214],
          [-0.1340, -0.1405, -0.0276,  ..., -1.7934, -2.3689, -2.3425],
          ...,
          [ 0.0790, -0.0141, -0.1522,  ..., -0.5747, -0.3621, -0.4219],
          [ 0.6042,  1.1715,  1.6839,  ...,  1.5596,  1.7559,  1.8693],
          [ 0.0784,  0.1942,  0.4039,  ...,  0.7810,  0.7711,  1.1252]]],
        device='cuda:0', grad_fn=<TransposeBackward0>),
 tensor([[[-1.1990, -1.5487, -0.5473,  ...,  0.3762,  0.8704,  0.3902],
          [ 0.0478, -1.5426, -1.0180,  ..., -2.1918, -0.0705, -1.1214],
          [-0.6363, -0.1405, -0.0276,  ..., -2.8669, -2.7566, -2.3425],
          ...,
          [-0.3257, -1.4200, -0.1522,  ...,  0.5995, -0.3621, -0.4219],
          [ 0.6042,  1.1715,  1.9447,  ...,  1.3170,  0.1206,  1.3306],
          [-0.8233, -1.8485, -1.4424,  ...,  0.7810,  1.3342,  2.1612]]],
        device='cuda:0', grad_fn=<AddBackward0